<a href="https://colab.research.google.com/github/edramos-lab/Datasets/blob/main/Image_Recognition_for_Baseplates_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install wandb -qq

     |████████████████████████████████| 1.7 MB 29.5 MB/s 
     |████████████████████████████████| 180 kB 63.0 MB/s 
     |████████████████████████████████| 139 kB 56.6 MB/s 
     |████████████████████████████████| 97 kB 7.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [5]:
 #Standard Libraries and OpenCV in order to manage screen
import cv2
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

#import ImageAumgentation Library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Architecture and Optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

#Hyperparameters & Tensorflow library
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import tensorflow as tf
import matplotlib.text as mpl_text
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

#Models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

#Reporting
import seaborn as sn
from sklearn.metrics import classification_report,confusion_matrix
import logging
import threading
import time
import datetime as dt
from wandb.keras import WandbCallback

In [6]:
from google.colab import drive

drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [7]:
cd '/content/gdrive/MyDrive'

/content/gdrive/MyDrive


In [8]:
TRAINING_DATA_DIR = 'sample_data/trainingImages_original'

In [14]:
#%%
datagen_kwargs = dict(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,validation_split=0.2)

"""1.3 Images for training:"""

'1.3 Images for training:'

In [15]:
#%%
def Create_Images_for_training(shape_size_W, shape_size_H):

  IMAGE_SHAPE=(shape_size_W, shape_size_H)
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
  train_generator = train_datagen.flow_from_directory(
  TRAINING_DATA_DIR,
  subset="training",
  shuffle=False,
  target_size=IMAGE_SHAPE)
  for image_batch, label_batch in train_generator:
    break
  print(image_batch.shape, label_batch.shape)
  image_batch_train, label_batch_train = next(iter(train_generator))
  valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
  valid_generator = valid_datagen.flow_from_directory(
  TRAINING_DATA_DIR,
  subset="validation",
  shuffle=False,
  target_size=IMAGE_SHAPE
  )
  val_image_batch, val_label_batch = next(iter(valid_generator))
  print("Image batch shape: ", image_batch_train.shape)
  print("Label batch shape: ", label_batch_train.shape)
  dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
  dataset_labels = np.array([key.title() for key, value in dataset_labels])
  print(dataset_labels)
  
  print(label_batch[0:5])
  print(train_generator.class_indices)
  labels = '\n'.join(sorted(train_generator.class_indices.keys()))
  with open('labels.txt', 'w') as f:
    f.write(labels)
  return train_generator,valid_generator,dataset_labels

train_generator,valid_generator,dataset_labels=Create_Images_for_training(640,480)
#%%
print(train_generator)

Found 9600 images belonging to 6 classes.
(32, 640, 480, 3) (32, 6)
Found 2400 images belonging to 6 classes.
Image batch shape:  (32, 640, 480, 3)
Label batch shape:  (32, 6)
['Class1' 'Class2' 'Class3' 'Class4' 'Class5' 'Class6']
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
{'Class1': 0, 'Class2': 1, 'Class3': 2, 'Class4': 3, 'Class5': 4, 'Class6': 5}


In [16]:
train_generator,valid_generator,dataset_labels=Create_Images_for_training(640,480)

Found 9600 images belonging to 6 classes.
(32, 640, 480, 3) (32, 6)
Found 2400 images belonging to 6 classes.
Image batch shape:  (32, 640, 480, 3)
Label batch shape:  (32, 6)
['Class1' 'Class2' 'Class3' 'Class4' 'Class5' 'Class6']
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
{'Class1': 0, 'Class2': 1, 'Class3': 2, 'Class4': 3, 'Class5': 4, 'Class6': 5}


In [17]:
import numpy as np
import plotly.graph_objs as go

import keras
from wandb.keras import WandbCallback
from sklearn.metrics import confusion_matrix

class WandbClassificationCallback(WandbCallback):

    def __init__(self, monitor='val_loss', verbose=0, mode='auto',
                 save_weights_only=False, log_weights=False, log_gradients=False,
                 save_model=True, training_data=None, validation_data=None,
                 labels=[], data_type=None, predictions=1, generator=None,
                 input_type=None, output_type=None, log_evaluation=False,
                 validation_steps=None, class_colors=None, log_batch_frequency=None,
                 log_best_prefix="best_", log_confusion_matrix=False, confusion_examples=0):
        
        super().__init__(monitor=monitor,
                        verbose=verbose, 
                        mode=mode,
                        save_weights_only=save_weights_only,
                        log_weights=log_weights,
                        log_gradients=log_gradients,
                        save_model=save_model,
                        training_data=training_data,
                        validation_data=validation_data,
                        labels=labels,
                        data_type=data_type,
                        predictions=predictions,
                        generator=generator,
                        input_type=input_type,
                        output_type=output_type,
                        log_evaluation=log_evaluation,
                        validation_steps=validation_steps,
                        class_colors=class_colors,
                        log_batch_frequency=log_batch_frequency,
                        log_best_prefix=log_best_prefix)
                        
        self.log_confusion_matrix = log_confusion_matrix
        self.confusion_examples = confusion_examples
               
    def on_epoch_end(self, epoch, logs={}):
        if self.generator:
            self.validation_data = next(self.generator)

        if self.log_weights:
            wandb.log(self._log_weights(), commit=False)

        if self.log_gradients:
            wandb.log(self._log_gradients(), commit=False)
        
        if self.log_confusion_matrix:
            if self.validation_data is None:
                wandb.termwarn(
                    "No validation_data set, pass a generator to the callback.")
            elif self.validation_data and len(self.validation_data) > 0:
                wandb.log(self._log_confusion_matrix(), commit=False)

        if self.input_type in ("image", "images", "segmentation_mask") or self.output_type in ("image", "images", "segmentation_mask"):
            if self.validation_data is None:
                wandb.termwarn(
                    "No validation_data set, pass a generator to the callback.")
            elif self.validation_data and len(self.validation_data) > 0:
                wandb.log({"examples": self._log_images(
                    num_images=self.predictions)}, commit=False)

        wandb.log({'epoch': epoch}, commit=False)
        wandb.log(logs, commit=True)

        self.current = logs.get(self.monitor)
        if self.current and self.monitor_op(self.current, self.best):
            if self.log_best_prefix:
                wandb.run.summary["%s%s" % (self.log_best_prefix, self.monitor)] = self.current
                wandb.run.summary["%s%s" % (self.log_best_prefix, "epoch")] = epoch
                if self.verbose and not self.save_model:
                    print('Epoch %05d: %s improved from %0.5f to %0.5f' % (
                        epoch, self.monitor, self.best, self.current))
            if self.save_model:
                self._save_model(epoch)
            self.best = self.current
        
    def _log_confusion_matrix(self):
        x_val = self.validation_data[0]
        y_val = self.validation_data[1]
        y_val = np.argmax(y_val, axis=1)
        y_pred = np.argmax(self.model.predict(x_val), axis=1)

        confmatrix = confusion_matrix(y_pred, y_val, labels=range(len(self.labels)))
        confdiag = np.eye(len(confmatrix)) * confmatrix
        np.fill_diagonal(confmatrix, 0)

        confmatrix = confmatrix.astype('float')
        n_confused = np.sum(confmatrix)
        confmatrix[confmatrix == 0] = np.nan
        confmatrix = go.Heatmap({'coloraxis': 'coloraxis1', 'x': self.labels, 'y': self.labels, 'z': confmatrix,
                                 'hoverongaps':False, 'hovertemplate': 'Predicted %{y}<br>Instead of %{x}<br>On %{z} examples<extra></extra>'})

        confdiag = confdiag.astype('float')
        n_right = np.sum(confdiag)
        confdiag[confdiag == 0] = np.nan
        confdiag = go.Heatmap({'coloraxis': 'coloraxis2', 'x': self.labels, 'y': self.labels, 'z': confdiag,
                               'hoverongaps':False, 'hovertemplate': 'Predicted %{y} just right<br>On %{z} examples<extra></extra>'})

        fig = go.Figure((confdiag, confmatrix))
        transparent = 'rgba(0, 0, 0, 0)'
        n_total = n_right + n_confused
        fig.update_layout({'coloraxis1': {'colorscale': [[0, transparent], [0, 'rgba(180, 0, 0, 0.05)'], [1, f'rgba(180, 0, 0, {max(0.2, (n_confused/n_total) ** 0.5)})']], 'showscale': False}})
        fig.update_layout({'coloraxis2': {'colorscale': [[0, transparent], [0, f'rgba(0, 180, 0, {min(0.8, (n_right/n_total) ** 2)})'], [1, 'rgba(0, 180, 0, 1)']], 'showscale': False}})

        xaxis = {'title':{'text':'y_true'}, 'showticklabels':False}
        yaxis = {'title':{'text':'y_pred'}, 'showticklabels':False}

        fig.update_layout(title={'text':'Confusion matrix', 'x':0.5}, paper_bgcolor=transparent, plot_bgcolor=transparent, xaxis=xaxis, yaxis=yaxis)
        
        return {'confusion_matrix': wandb.data_types.Plotly(fig)}


#2. ConvNet Architecture

In [18]:
!pip install wandb

In [19]:
hyperparameter_defaults = dict(
  dropout_1 = 0.25,
  dropout_2 = 0.5,
  learning_rate = 0.001,
  epochs = 10,
  batch_size = 16,
  conv2d_1_filters = 32,
  conv2d_1_kernel_size= 3,
  conv2d_2_filters = 32,
  conv2d_2_kernel_size = 3,
  dense_1_units = 128,
  pool_1_size = 2,
)

In [20]:
import wandb
run = wandb.init(project="ImageRecognition1", entity="edramos")
if run is None:
    raise ValueError("Wandb didn't initialize properly")
config = wandb.config

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [21]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 10,
  "batch_size": 16
}

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

batchSize = 16
model = Sequential([
       
  Conv2D(batchSize, (3, 3), activation='relu',input_shape=(640, 480,3)),
  MaxPooling2D(2,2),
  Conv2D(32,(3,3),activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(64,(3,3),activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(128,(3,3),activation='relu'),
  Flatten(),
  Dense(256,activation='relu'),
  Dropout(0.2),   
  Dense(6, activation='softmax')
        
 ])
     

#3. Optimization and Compiling

In [22]:
es = EarlyStopping()

optimizer=tf.keras.optimizers.Adam(lr=0.001)
es = EarlyStopping()
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [23]:

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)


In [24]:
!nvidia-smi 

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [25]:

with tf.device("/gpu:0"):
 

  # Optional
  
  batch_size=16
  hist = model.fit(
          train_generator, 
          epochs=10,
          verbose=1,
          steps_per_epoch=steps_per_epoch,
          validation_data=valid_generator,
          validation_steps=val_steps_per_epoch, callbacks=[WandbClassificationCallback(log_confusion_matrix=True, validation_data=valid_generator, labels=list(range(6)))]).history




Epoch 1/10
  2/300 [..............................] - ETA: 1:30:48 - loss: 28.6056 - accuracy: 0.0781

KeyboardInterrupt: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa66bb9b390>> (for post_run_cell):


Exception: ignored

In [ ]:
STEP_SIZE_TEST=1#valid_generator.n//valid_generator.batch_size
valid_generator.reset()
pred=model.predict_generator(valid_generator,
steps=32,
verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:1289: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=valid_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

ValueError: ignored